# Data Preparation

## Import the Data

In [1]:
import pandas as pd


In [2]:
try:
    df = pd.read_csv("data/data_deals.csv")
except Exception:
    print("Seems we have a problem")


In [3]:
print("File Columns are:", df.columns)
print("Number of rows:", df.shape[0])

File Columns are: Index(['_id', 'sale_day', 'address', 'property_type', 'rooms_number', 'floor',
       'build_year', 'building_mr', 'sold_part', 'full_price', 'city'],
      dtype='object')
Number of rows: 517569


## Data Modification and CleanUps

### Date Formating
Convert all the dates strings into Pandas datetime object

In [4]:
# sale_day "05-06-2016"
# build_year "2017"
df['sale_day'] = pd.to_datetime(df['sale_day'], format="%d-%m-%Y")
df['build_year'] = pd.to_datetime(df['build_year'], format="%Y")


### Removing Un-necessary Data
Remvove all properties that are:
1. Price is not in NIS or N/A
2. build_year is before 1945 (Data before that is not relevant)
3. Floor is not N/A
4. Property Type is only euqal to: דירה בבית קומות, דירת מגורים

In [5]:
# remove rows that their build_year is before 1945
from_year = 1945
df = df.loc[(df['build_year'].dt.year >= from_year)]
df['build_year'] = df['build_year'].apply(lambda x: x.strftime('%Y')).astype(int)
print("Number of rows:", df.shape[0])


Number of rows: 501996


In [6]:
# remove rows that their property_type is not in list
# valid_values = ["דירה בבית קומות", "דירת מגורים"]
# df = df[df['property_type'].isin(valid_values)]
# print("Number of rows:", df.shape[0])

In [7]:
# List unique values in the df['name'] column
cols = ['full_price', 'sold_part', 'floor']
for col in cols:
    try:
        df.replace({col: {'₪': '',
                          '\$': '',
                          'קרקע': 0,
                          'לא צוין מחיר': None,
                          'לא צוין': None,
                          ',': '',
                          '%': '',
                          ' ': ''}}, regex=True, inplace=True)
    except TypeError:
        # in case of double replacement
        pass


In [8]:
# keep only floors that are with values
df = df[pd.notnull(df['floor'])]
print("Number of rows:", df.shape[0])

# format values to their types
df['rooms_number'] = df['rooms_number'].astype(float)
df['floor'] = df['floor'].astype(int)
df['building_mr'] = df['building_mr'].astype(int)
df['full_price'] = df['full_price'].astype(int)


Number of rows: 499717


In [9]:
# remove properties that their:
#     mr is more than 200
#     floor is more than 20
#     rooms is more than 7
from_mr = 200
from_floor = 20
from_room = 7.0

df = df.loc[(df['building_mr'] <= from_mr) & (df['floor'] <= from_floor) & (df['rooms_number'] <= from_room)]
print("Number of rows:", df.shape[0])


Number of rows: 491412


## Calculate the final price column
There are cases that only part of the property was bought (value by percentage)

In [10]:
# # format sold_part to full_price
# df['sold_part'] = df['sold_part'].astype(float)
# print(df.shape[0] - df['sold_part'].isnull().sum())
# df['sold_part'].fillna(100, inplace=True)
# df['final_price'] = (100 / df['sold_part']) * df['full_price']

df = df.loc[df['sold_part'].isnull()]
df['final_price'] = df['full_price']
df['final_price'] = df['final_price'].astype(int)
df.drop(columns=['_id', 'sold_part', 'full_price'], inplace=True)
print("Number of rows:", df.shape[0])


Number of rows: 423301


## Split Sale Date to Year and Month columns

In [11]:
df['sale_day_year'] = df['sale_day'].apply(lambda x: x.strftime('%Y')).astype(int)
df['sale_day_month'] = df['sale_day'].apply(lambda x: x.strftime('%m')).astype(int)

## Split Address to street and street number

In [12]:
split_data = df['address'].str.rsplit(n=1, expand=True)
df['street'] = split_data[0]
df['street_number'] = split_data[1]


## Merge neighborhood value from a different mapping

In [13]:
try:
    nmap = pd.read_csv("data/map_address_neighborhood.csv")
    print(nmap.columns)
    print(nmap.shape)
except Exception:
    print("Seems we have a problem")


df = pd.merge(df, nmap,  how='inner', on=['city', 'street'])


Index(['street', 'city', 'neighborhood', 'address_area',
       'address_neighborhood'],
      dtype='object')
(10971, 5)


## Save results to final file

In [14]:
# Sort the rows of dataframe by column 'final_price' inplace
df.sort_values(by='final_price' , inplace=True, ascending=False)

In [15]:
df.to_csv('data/data_deals_done.csv', encoding='utf-8', index=False)